### API Key 불러오기

In [1]:
import os
from dotenv import load_dotenv

# API Key 환경변수 설정
load_dotenv()

True

### 유튜브 동영상 내용 요약

In [2]:
import requests
import os

def summarize_contents(contents_url, target_language):
    
    TOKEN = os.getenv("KAGI_API_KEY") 

    base_url = 'https://kagi.com/api/v0/summarize'
    params = {
        "url": contents_url,
        "summary_type": "summary",
        "engine": "muriel",
        "target_language": target_language
    }
    headers = {'Authorization': f'Bot {TOKEN}'}

    response = requests.get(base_url, headers=headers, params=params)
    summary = response.json()['data']['output']
    
    return summary

In [3]:
import textwrap

contents_url = "https://www.youtube.com/watch?v=Ks-_Mh1QhMc"
target_language = "KO" # 요약할 언어를 한국어로 지정

try:
    summary = summarize_contents(contents_url, target_language)

    print("[콘텐츠 URL]", contents_url)
    print(textwrap.shorten(summary, 150 ,placeholder=' [..이하 생략..]')) # 축약 출력
    # print(summary) # 전체 출력
except:
    print("해당 URL의 내용을 요약하지 못했습니다. 다시 시도해 주세요.")

[콘텐츠 URL] https://www.youtube.com/watch?v=Ks-_Mh1QhMc
에이미 커디(Amy Cuddy)의 TED 강연 "당신의 신체 언어가 당신을 만듭니다(Your Body Language May Shape Who You Are)"에서, 그녀는 신체 언어가 우리가 자신을 어떻게 인식하고 타인이 우리를 어떻게 [..이하 생략..]


In [ ]:
import textwrap

contents_urls = ["https://www.youtube.com/watch?v=arj7oStGLkU",
                 "https://www.youtube.com/watch?v=lmyZMtPVodo",
                 "https://www.youtube.com/watch?v=JdwWgw4fq7I"]
                 
target_language = "KO" # 요약할 언어를 한국어로 지정

for contents_url in contents_urls:
    try:
        summary = summarize_contents(contents_url, target_language)
        print("[콘텐츠 URL]", contents_url)
        print(textwrap.shorten(summary, 150 ,placeholder=' [..이하 생략..]')) # 축약 출력
        # print(summary) # 전체 출력
    except:
        print("해당 URL의 내용을 요약하지 못했습니다. 다시 시도해 주세요.")        

### 비디오, 오디오 파일의 음성을 텍스트로 추출

In [ ]:
import openai
from pathlib import Path
from openai import OpenAI

def audio_transcribe(input_path, resp_format="text", lang="en"):
    client = OpenAI()

    with open(input_path, "rb") as f: # 음성 파일 열기
        # 음성 추출  
        transcript_response = client.audio.transcriptions.create(
            model = "gpt-4o-mini-transcribe",
            file = f,
            language = lang,            
            response_format = resp_format # 추출할 텍스트 형식 지정
        )
    
    # 음성을 텍스트로 추출한 후에 텍스트 파일로 저장
    path = Path(input_path)
    output_file = path.with_suffix('.txt') if resp_format == "text" else path.with_suffix('.srt')
        
    with open(output_file, 'w', encoding='utf-8') as f:
        if isinstance(transcript_response, str):
            f.write(transcript_response)
        else:
            f.write(transcript_response.text)

    return transcript_response, output_file

In [ ]:
audio_file = "download/youtube_video_file.mp3"
print(f"[음성 파일 경로] {audio_file}\n")

r_format = "srt" # 출력 형식을 srt로 지정
transcript, output_path = audio_transcribe(audio_file, r_format)    

print(f"- [텍스트 추출 형식] {r_format}")
print(f"- [출력 파일] {output_path.name}")
print(f"- [음성 추출 결과(일부 출력)]\n{transcript[:137]}") # 추출 결과 출력(일부)
# print(f"- [음성 추출 결과]\n{transcript}") # 추출 결과 출력(전체)

### 동영상 자막 번역

In [5]:
import deepl
from pathlib import Path

def transplate_text_file(input_path, t_lang="KO"):
    
    # 원본 파일 읽기
    with open(input_path, encoding='utf-8') as f:
        text = f.read()

    # 번역 수행
    auth_key = os.getenv("DeepL_API_KEY") # Deepl 인증 키
    translator = deepl.Translator(auth_key) # translator 객체를 생성
    result = translator.translate_text(
        text,               # 원본(소스) 텍스트
        target_lang = "KO"  # 대상(타깃) 언어 코드  
    ) 
    
    # 번역 파일 쓰기
    path = Path(input_path)
    output_file = f"{path.parent}/{path.stem}_번역_{t_lang}{path.suffix}"
    
    output_path = Path(output_file)    

    with open(output_file, 'w', encoding='utf-8') as f:
        f.write(result.text)   

    return output_path

In [6]:
input_file = "download/aCbqE30AfUs.srt"

translated_file = transplate_text_file(input_file)
print("- 번역 파일:", translated_file.name)

- 번역 파일: aCbqE30AfUs_번역_KO.srt
